# Advanced Lane Detection

## (using Deep Learning)




### Import TensorFlow and Keras libraries

We will start by importing all the necessary libraries for Keras and TensorFlow. Apart from these libraries, we will also import some other libraries such as NumPy, Glob, cv2, csv, and MatPlotLib.


In [ ]:
import os
import csv
import cv2
import numpy as np
import keras
import tensorflow as tf
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import random
import math

from scipy import ndimage

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Cropping2D
from keras.layers import Dropout

import sklearn
from sklearn.model_selection import train_test_split

import GPUtil as GPU

## GPU information

Here, we will print some information related to GPU memory capacity. The user can accordingly make modifications to the code in case of more/less capacity.

In [ ]:
import GPUtil as GPU
GPUs = GPU.getGPUs()
gpu = GPUs[0]
print("Total Available GPU memory: {} MB".format(gpu.memoryTotal))
print("Used GPU memory: {} MB".format(gpu.memoryUsed))
print("Total Free GPU memory: {} MB".format(gpu.memoryFree))

## Check library versions

Here, we print the versions of tensorflow, keras and OpenCV versions to compare compatibility.

In [ ]:
# printing installed tensorflow/keras/OpenCV versions
print("Current OpenCV version:", cv2.__version__)
print("Using TensorFlow version:", tf.__version__)
print("Using Keras version:", keras.__version__)

## Dataset building and Augmentation

Here, we load the data from the *driving_log.csv* file and if required, augment the data by flipping the images in the opposite direction.

In [ ]:
# 1. Start of dataset building
## Read the csv file containing saved model data

lines = []
with open('./data/model_data.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

# empty list to save images        
binary_images = []

# lists to save polynomial measurements for left and right lanes

left_measurements_a = []
left_measurements_b = []
left_measurements_c = []
right_measurements_a = []
right_measurements_b = []
right_measurements_c = []

# directory for accessing binary images
directory = '../data_for_lane_detection/binary_images/'
directory = '../data_for_lane_detection/lanelines_images/'
directory = '../data_for_lane_detection/transformed_images/'

# setting data augmentation activation to True/False
augment = True

if (augment):
    print("Performing data augmentation by flipping the images ...\n")

for line in lines:

    # reading filesnames for binary images and lanelines images
    fname_binary_img = directory + line[0].split('/')[-1]
    fname_lanelines_img = directory + line[1].split('/')[-1]
    fname_transformed_img = directory + line[2].split('/')[-1]

    # reading images using opencv
    bin_img = mpimg.imread(fname_binary_img)
    laneline_img = mpimg.imread(fname_lanelines_img)
    
    # adding images to the lists
    images.append(fname_binary_img)
    images.append(fname_lanelines_img)
    
    # adding steering angles for center. left and right images

    left_measurements_a.append(float(line[3]))
    left_measurements_b.append(float(line[4]))
    left_measurements_c.append(float(line[5]))
    right_measurements_a.append(float(line[6]))
    right_measurements_b.append(float(line[7]))
    right_measurements_c.append(float(line[8]))

    # only if data augmentation is required
    if (augment):
        
        # this is where I perform data augmentation for images and angles
        binary_img_flipped = np.fliplr(fname_binary_img)
        images.append(binary_img_flipped)
        

        laneline_img_flipped = np.fliplr(fname_laneline_img)
        images.append(lanelines_img_flipped)


        transformed_img_flipped = np.fliplr(fname_transformed_img)
        images.append(transformed_img_flipped)
        
        # flip the measurements
        left_measurements_a_flip = -(float(line[3]))
        left_measurements_b_flip = -(float(line[4]))
        left_measurements_c_flip = -(float(line[5]))
        right_measurements_a_flip = -(float(line[6]))
        right_measurements_b_flip = -(float(line[7]))
        right_measurements_c_flip = -(float(line[8]))
        
        # add the flipped measurements to the current measurements lists
        left_measurements_a.append(left_measurements_a_flip)
        left_measurements_b.append(left_measurements_b_flip)
        left_measurements_c.append(left_measurements_c_flip)
        right_measurements_a.append(right_measurements_a_flip)
        right_measurements_b.append(right_measurements_b_flip)
        right_measurements_c.append(right_measurements_c_flip)


# define arrays for datasets
X_train = np.array(images)
steering_angles = np.array(measurements)